In [5]:
from google.colab import drive
drive.mount('./gdrive', force_remount=True)

Mounted at ./gdrive


In [10]:
!pip install torch@https://download.pytorch.org/whl/cu111/torch-1.10.0%2Bcu111-cp37-cp37m-linux_x86_64.whl
!pip install tensorflow==2.0.3
!pip install pytorch-lightning==1.3.4
!pip install transformers==4.10.0
!pip install konlpy==0.6.0
!pip install Korpora==0.2.0

tcmalloc: large alloc 2137645056 bytes == 0x563870b16000 @  0x7f636488e1e7 0x56386e448518 0x56386e412d17 0x56386e5921f5 0x56386e52c8f8 0x56386e4165d2 0x56386e4f597e 0x56386e416229 0x56386e508b3d 0x56386e48a458 0x56386e4179da 0x56386e486108 0x56386e4179da 0x56386e486108 0x56386e4851c0 0x56386e41836c 0x56386e418571 0x56386e487633 0x56386e48502f 0x56386e41836c 0x56386e418571 0x56386e487633 0x56386e48502f 0x56386e41836c 0x56386e418571 0x56386e487633 0x56386e4851c0 0x56386e41836c 0x56386e418571 0x56386e487633 0x56386e48502f
tcmalloc: large alloc 2137645056 bytes == 0x5638f01b4000 @  0x7f636488e1e7 0x56386e448518 0x56386e412d17 0x56386e52a297 0x56386e41634c 0x56386e508b3d 0x56386e48a458 0x56386e4179da 0x56386e486108 0x56386e4179da 0x56386e486108 0x56386e4851c0 0x56386e41836c 0x56386e418571 0x56386e487633 0x56386e48502f 0x56386e41836c 0x56386e418571 0x56386e487633 0x56386e48502f 0x56386e41836c 0x56386e418571 0x56386e487633 0x56386e4851c0 0x56386e41836c 0x56386e418571 0x56386e487633 0x56386e48

     |████████████████████████████████| 57 kB 4.1 MB/s 
     |████████████████████████████████| 96 kB 4.6 MB/s 
  Attempting uninstall: xlrd
    Found existing installation: xlrd 1.1.0
    Uninstalling xlrd-1.1.0:
      Successfully uninstalled xlrd-1.1.0


In [11]:
import torch
from pytorch_lightning import Trainer, LightningModule
from pytorch_lightning.callbacks import ModelCheckpoint
from transformers import PreTrainedModel, BertTokenizer, BertConfig, BertForSequenceClassification
from transformers.optimization import AdamW
from Korpora import Korpora
import os, csv, re
from dataclasses import dataclass
from typing import List, Optional
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.optim.lr_scheduler import ExponentialLR
from sklearn.model_selection import train_test_split
import pandas as pd 

In [12]:
!pwd

/content


In [15]:
trainDataDir = './gdrive/MyDrive/chatbot_model'
trainDataPath = os.path.join(trainDataDir, 'total_train.csv')
trainDataPath

'./gdrive/MyDrive/chatbot_model/total_train.csv'

In [16]:
trainCSVData = list(csv.reader(open(trainDataPath, "r", encoding="utf-8"), delimiter=",", quotechar='"'))[1:-1]
trainCSVData[:5]

[['지금 배달되나요?', '1', '0', '0', '0', '0', '0', '0', '0', '0'],
 ['아 네 배달됩니다', '1', '0', '0', '0', '0', '0', '0', '0', '0'],
 ['짬뽕류는 어떤 게 있나요? 잘 나가는 짬뽕 있나요?', '1', '0', '0', '0', '0', '0', '0', '0', '0'],
 ['특해물 짬뽕도 있고 전복 새우 짬뽕도 있고 해물 종류도 새우 홍합 전복 없는 게 없습니다',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0'],
 ['전복 들어가는 거는 특해물 짬뽕 시켜야 돼요?', '1', '0', '0', '0', '0', '0', '0', '0', '0']]

In [17]:
trainCSVData[228363]

['아까 요 제품 이 저 는 좀 나은 거 같은 데 가격 은 똑같 아예 ?',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0']

위는 쉼표가 포함된 문장도 잘 불러와졌는지 한번 보려고 출력해본 것

In [18]:
train, test = train_test_split(trainCSVData, test_size=0.3)
'train', train[:2], len(train), 'test', test[:2], len(test)

('train',
 [['이 보라색 티 는 얼마 예요', '0', '1', '0', '0', '0', '0', '0', '0', '0'],
  ['결재해 주세요', '0', '0', '0', '0', '0', '0', '1', '0', '0']],
 253155,
 'test',
 [['저희는 S/M/L/XL/XXL까지 있습니다', '0', '1', '0', '0', '0', '0', '0', '0', '0'],
  ['저 신발 은 신 을 때 뒤에 신발 주걱 이나 그런거 해 서 신 을 필욘 없고 ?',
   '0',
   '1',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0']],
 108496)

In [19]:
@dataclass
class ClassificationDataFormat:
    text_a: str
    label: Optional[int] = None

In [20]:
ClassificationDataFormat(text_a=train[0][0], label=train[0][1:].index('1'))

ClassificationDataFormat(text_a='이 보라색 티 는 얼마 예요', label=1)

In [21]:
[int(label) for label in train[0][1:]]

[0, 1, 0, 0, 0, 0, 0, 0, 0]

In [22]:
trainDataFormatList = []

In [23]:
for data in train:
    trainDataFormatList.append(ClassificationDataFormat(text_a=data[0], label=data[1:].index('1')))
trainDataFormatList[:5]

[ClassificationDataFormat(text_a='이 보라색 티 는 얼마 예요', label=1),
 ClassificationDataFormat(text_a='결재해 주세요', label=6),
 ClassificationDataFormat(text_a='사이즈 230 으로 포 장해 주 세 요', label=1),
 ClassificationDataFormat(text_a='콜드브루 있 나 요 ?', label=5),
 ClassificationDataFormat(text_a='여기 상가 임대 나온 게 있어요', label=8)]

In [24]:
testDataFormatList = []

In [25]:
for data in test:
    testDataFormatList.append(ClassificationDataFormat(text_a=data[0], label=data[1:].index('1')))
testDataFormatList[:5]

[ClassificationDataFormat(text_a='저희는 S/M/L/XL/XXL까지 있습니다', label=1),
 ClassificationDataFormat(text_a='저 신발 은 신 을 때 뒤에 신발 주걱 이나 그런거 해 서 신 을 필욘 없고 ?', label=1),
 ClassificationDataFormat(text_a='보여주신 매물 계약 어떻게 할까요', label=8),
 ClassificationDataFormat(text_a='130g에 5천 원이요', label=3),
 ClassificationDataFormat(text_a='피부 트러블 은 없으 신 가요 ?', label=3)]

In [26]:
@dataclass
class ARGS:
    pretrained_model_name: str = 'beomi/kcbert-base'
    batch_size: int = 16
    learning_rate: float = 5e-5
    max_seq_length: int = 64
    epochs: int = 3
    tpu_cores: int = 0
    downstream_task_name: str = "chat-type-classification"
    cpu_workers: int = 7
    downstream_model_dir: str = './gdrive/MyDrive/chatbot_model'
args = ARGS()

In [27]:
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name, #pre-trained model
    do_lower_case=False
)
tokenizer

Downloading:   0%|          | 0.00/250k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

PreTrainedTokenizer(name_or_path='beomi/kcbert-base', vocab_size=30000, model_max_len=300, is_fast=False, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [ ]:
batchEncoding = tokenizer(
    [data.text_a for data in trainDataFormatList],    
    max_length=args.max_seq_length, # 128
    padding="max_length", # 최대 길이 만큼 패딩
    truncation=True # 길이 오버시 자름 
)

In [ ]:
batchEncoding.keys()

In [ ]:
input_sample = {k: batchEncoding[k][0] for k in batchEncoding}

In [ ]:
@dataclass
class ClassificationFeatures:
    input_ids: List[int]
    attention_mask: Optional[List[int]] = None
    token_type_ids: Optional[List[int]] = None
    label: Optional[List[int]] = None

In [ ]:
feature_sample = ClassificationFeatures(**input_sample, label=trainDataFormatList[0].label)
feature_sample

In [ ]:
train_feature = []

In [ ]:
for idx in range(len(trainDataFormatList)):
    input_tokenized = {k: batchEncoding[k][idx] for k in batchEncoding}
    feature = ClassificationFeatures(**input_tokenized, label=trainDataFormatList[idx].label)
    train_feature.append(feature)
train_feature[:3]

In [ ]:
for idx, data in enumerate(trainDataFormatList[:3]):
    print(f"#{idx}: {data.text_a}")
    token = " /".join(tokenizer.convert_ids_to_tokens(train_feature[idx].input_ids))
    print(f"token: {token}\n========\n")

In [ ]:
batchEncoding_for_test = tokenizer(
    [data.text_a for data in testDataFormatList],    
    max_length=args.max_seq_length, # 128
    padding="max_length", # 최대 길이 만큼 패딩
    truncation=True # 길이 오버시 자름 
)

In [ ]:
test_feature = []

In [ ]:
for idx in range(len(testDataFormatList)):
    input_tokenized = {k: batchEncoding_for_test[k][idx] for k in batchEncoding}
    feature = ClassificationFeatures(**input_tokenized, label=testDataFormatList[idx].label)
    test_feature.append(feature)
test_feature[:3]

In [ ]:
for idx, data in enumerate(testDataFormatList[:3]):
    print(f"#{idx}: {data.text_a}")
    token = " /".join(tokenizer.convert_ids_to_tokens(test_feature[idx].input_ids))
    print(f"token: {token}\n========\n")

In [ ]:
def data_collator(features):
    batch = {}
    batch["input_ids"] = torch.tensor([feature.input_ids for feature in features], dtype=torch.long)
    batch["attention_mask"] = torch.tensor([feature.attention_mask for feature in features], dtype=torch.long)
    batch["token_type_ids"] = torch.tensor([feature.token_type_ids for feature in features], dtype=torch.long)
    batch["labels"] = torch.tensor([feature.label for feature in features], dtype=torch.long)
    return batch

In [ ]:
data_collator(train_feature)

In [ ]:
trainDataLoader = DataLoader(
    train_feature,
    batch_size=args.batch_size,
    sampler=RandomSampler(train_feature, replacement=False),
    collate_fn=data_collator,
    drop_last=False,
    num_workers=args.cpu_workers
)

In [ ]:
testDataLoader = DataLoader(
    test_feature,
    batch_size=args.batch_size,
    sampler=RandomSampler(test_feature, replacement=False),
    collate_fn=data_collator,
    drop_last=False,
    num_workers=args.cpu_workers
)

In [ ]:
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=9,
)

In [ ]:
model = BertForSequenceClassification.from_pretrained(
    args.pretrained_model_name,
    config=pretrained_model_config
)

In [ ]:
checkpointPath = args.downstream_model_dir
os.makedirs(checkpointPath, exist_ok=True)

In [ ]:
checkpoint_callback = ModelCheckpoint(
        dirpath=checkpointPath,
        save_top_k=1,
        monitor="val_loss",
        mode="min",
        filename='{epoch}-{val_loss:.2f}',
    )

In [ ]:
trainer = Trainer(
    max_epochs=args.epochs,
    fast_dev_run=False,
    num_sanity_val_steps=0,
    callbacks=[checkpoint_callback],
    default_root_dir=checkpointPath,
    gpus=torch.cuda.device_count() if torch.cuda.is_available() else None,
#     resume_from_checkpoint=f'{checkpointPath}/epoch=0-val_loss=0.62.ckpt'
)

In [ ]:
def accuracy(preds, labels, ignore_index=None):
    with torch.no_grad():
        assert preds.shape[0] == len(labels)
        correct = torch.sum(preds == labels)
        total = torch.sum(torch.ones_like(labels))
        if ignore_index is not None:
            # 모델이 맞춘 것 가운데 ignore index에 해당하는 것 제외
            correct -= torch.sum(torch.logical_and(preds == ignore_index, preds == labels))
            # accuracy의 분모 가운데 ignore index에 해당하는 것 제외
            total -= torch.sum(labels == ignore_index)
    return correct.to(dtype=torch.float) / total.to(dtype=torch.float)

In [ ]:
class ClassificationTrainTask(LightningModule):
    def __init__(self, model: PreTrainedModel, args):
        super().__init__()
        self.model = model
        self.args = args
        
    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=self.args.learning_rate)
        scheduler = ExponentialLR(optimizer, gamma=0.9)
        return {
            'optimizer': optimizer,
            'scheduler': scheduler
        }
    
    def training_step(self, inputs, batch_idx):
#         print('train input', inputs['labels'].size())
#         print(inputs)
        outputs = self.model(**inputs)
#         print('train input', inputs['labels'].size(), outputs)
        predict = outputs.logits.argmax(dim=-1)
        labels = inputs['labels']
        acc = accuracy(predict, labels)
        self.log("loss", outputs.loss, prog_bar=False, logger=True, on_step=True, on_epoch=False)
        self.log("acc", acc, prog_bar=True, logger=True, on_step=True, on_epoch=False)
        return outputs.loss
        
    def validation_step(self, inputs, batch_idx):
#         print('inputs?', inputs)
        outputs = self.model(**inputs)
        predict = outputs.logits.argmax(dim=-1)
        labels = inputs["labels"]
        acc = accuracy(predict, labels)
        self.log("val_loss", outputs.loss, prog_bar=True, logger=True, on_step=False, on_epoch=True)
        self.log("val_acc", acc, prog_bar=True, logger=True, on_step=False, on_epoch=True)
        return outputs.loss

In [ ]:
task = ClassificationTrainTask(model, args)

In [ ]:
trainer.fit(
    task,
    train_dataloader=trainDataLoader,
    val_dataloaders=testDataLoader
)

In [ ]:
train_feature[:1], trainDataFormatList[:1]

([ClassificationFeatures(input_ids=[2, 9703, 8225, 1023, 4123, 5137, 9718, 11514, 12397, 8877, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], label=3)],
 [ClassificationDataFormat(text_a='혹시 여기 떡국떡 따로 구매 가능한가요', label=3)])

In [ ]:
dataCollactorSample = data_collator(train_feature[12:15])
dataCollactorSample

{'input_ids': tensor([[    2,  2980,  5088,  7968,  2972,  4169,  9718, 11514,  8021, 14046,
              3,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0],
         [    2,  1373,  3288,  2851, 15255, 21953,    32,     3,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,   

In [ ]:
outputs = model(**dataCollactorSample)
outputs

SequenceClassifierOutput(loss=tensor(2.0194, grad_fn=<NllLossBackward0>), logits=tensor([[-0.6597,  0.0737, -0.3774, -0.6632,  0.0953, -0.2092,  0.1090, -0.8112,
         -0.1802],
        [-0.5069,  0.1797, -0.2550, -0.5503,  0.3207, -0.2133,  0.1562, -0.6542,
         -0.0902],
        [-0.4643,  0.1082,  0.1898, -0.1973,  0.4369, -0.0938, -0.1061, -0.6765,
         -0.0262]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [ ]:
pred = outputs.logits.argmax(dim=-1)
pred, dataCollactorSample['labels']

(tensor([6, 4, 4]), tensor([6, 8, 1]))

In [ ]:
accuracy(outputs.logits.argmax(dim=-1), dataCollactorSample['labels'])

tensor(0.3333)

In [ ]:
pred.shape[0], len(dataCollactorSample['labels'])

(3, 3)

In [ ]:
correct = torch.sum(pred == dataCollactorSample['labels'])
correct

tensor(1)

맞은 결과들의 합

In [ ]:
total = torch.sum(torch.ones_like(dataCollactorSample['labels']))
total, torch.ones_like(dataCollactorSample['labels'])

(tensor(3), tensor([1, 1, 1]))

ones_like: 텐서 형태 안에 값을 모두 1로 변경 후 

In [ ]:
correct / total

tensor(0.3333)

In [ ]:
outputs.loss

tensor(2.0194, grad_fn=<NllLossBackward0>)